In [200]:
from oceanograpy.data.ship_ctd_tools import _ctd_edit, _ctd_tools
from oceanograpy.data import ctd
import importlib as imp
imp.reload(_ctd_edit)
imp.reload(ctd)

%matplotlib widget
D = ctd.ctds_from_cnv_dir('../example_data/cnv/atwain_cruise/')

Found 10 .cnv files in  "../example_data/cnv/atwain_cruise/".
Loaded 10 profiles from list of Datasets.
NOTE: It seems the input data already binned -> using preexisting binning.


Joining profiles together:   0%|          | 0/10 [00:00<?, ?it/s]

In [201]:
ctd.hand_remove_points(D)

TypeError: hand_remove_points.__init__() missing 2 required positional arguments: 'varnm' and 'station'

In [194]:
D

<xarray.Dataset>
Dimensions:             (PRES: 871, TIME: 10)
Coordinates:
  * PRES                (PRES) float64 3.0 4.0 5.0 6.0 ... 871.0 872.0 873.0
  * TIME                (TIME) float64 1.743e+04 1.743e+04 ... 1.743e+04
Data variables:
    TEMP1               (TIME, PRES) float64 nan 0.6721 -0.3421 ... nan nan nan
    PTEMP1              (TIME, PRES) float64 nan 0.672 -0.3422 ... nan nan nan
    CNDC1               (TIME, PRES) float64 nan 28.13 27.35 ... nan nan nan
    PSAL1               (TIME, PRES) float64 nan -66.94 -66.87 ... nan nan nan
    SIGTH1              (TIME, PRES) float64 nan 26.5 26.61 ... nan nan nan
    CHLA1_fluorescence  (TIME, PRES) float64 nan 1.597 1.749 ... nan nan nan
    SBE_FLAG            (TIME, PRES) float64 nan 0.0 0.0 0.0 ... nan nan nan nan
    STATION             (TIME) <U5 'AT280' 'AT281' 'AT282' ... 'AT288' 'AT289'
    LATITUDE            (TIME) float64 80.44 80.41 80.38 ... 80.04 80.03 80.02
    LONGITUDE           (TIME) float64 8.665 8.715 8.799 ... 10.75 10.9 11.0
    CRUISE              <U27 '!! CRUISE (Not assigned) !!'
Attributes:
    binned:          1 decibars (SBE software)
    history:         2017-09-24 to 2017-09-24: Data collection.\n2018-07-23: ...
    latitude:        80.0318
    longitude:       10.8973
    SBE_processing:  SBE SOFTWARE PROCESSING STEPS (extracted from .cnv file ...
    source_files:    E.g. AT288.HEX, AT288.XMLCON -> AT288.CNV
    featureType:     profile

In [150]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

In [189]:
def apply_offset(D):

    vars = _ctd_tools._get_profile_variables(D)
    #var_buttons = widgets.ToggleButtons(
    #options=vars,
    #description='Apply offset to:',
    #disabled=False,
    #button_style='', # 'success', 'info', 'warning', 'danger' or ''
   #           'Apply an offset to only this particular profile',],
     #icons=['check'] * 2
    #)

    var_buttons = widgets.RadioButtons(
        options=vars,
        description='Apply offset to:',
        disabled=False
    )
    
    station_buttons = widgets.ToggleButtons(
    options=['All stations', 'Single station →', ],
    description='Apply offset to:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Apply an offset only to the profile from this particular station', 
        'Apply an offset to every single profile in the dataset'],
    )

    value_textbox = widgets.Text(
    #value='Enter a value',
    placeholder='Enter a numerical value',
    disabled=False   
    )
    value_label = widgets.Label(value=f'Value of offset:')
    value_widget = widgets.VBox([value_label, value_textbox])


    # Function to update the description of value_textbox
    def update_description(change):
        selected_variable = change['new']
        # Assuming you have a function to get units based on the selected variable
        units = D[var_buttons.value].units
        value_label.value = f'Value of offset ({units}):'

    # Attach the update_description function to the observe method of var_buttons
    var_buttons.observe(update_description, names='value')

    # Increase the width of the description
    value_textbox.style.description_width = '150px'  # Adjust the width as needed
    


    # Dropdown for selecting a single profile
    station_dropdown = widgets.Dropdown(
        options=list(D.STATION.values),  # Assuming keys are profile names
        value=D.STATION.values[0],  # Set default value
        disabled=False,
        style={'description_width': 'initial'},  # Adjust width
    )
    
  # Align items to the flex-end to move the dropdown to the bottom of the HBox
    hbox_layout = widgets.Layout(align_items='flex-end')
    
    # Create an HBox with station_buttons and station_dropdown
    hbox_station = widgets.HBox([station_buttons, station_dropdown], layout=hbox_layout)

    apply_button = widgets.ToggleButton(
        value=False,
        description='Apply offset',
        disabled=False,
        button_style='success', # 'success', 'info', 'warning', 'danger' or ''
       # tooltip='Description',
       # icon='check' # (FontAwesome names without the `fa-` prefix)
        )

    exit_button = widgets.ToggleButton(
        value=False,
        description='Exit',
        disabled=False,
        button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
       # tooltip='Description',
       # icon='check' # (FontAwesome names without the `fa-` prefix)
        )
    
    def on_apply_button_click(change):
        if change['new']:
            if value_textbox.value != '':
    
                varnm_sel = var_buttons.value
                offset_value = float(value_textbox.value)
                units = D[varnm_sel].units
                var_attrs = D[varnm_sel].attrs
                
                if station_buttons.value == 'Single station →':
                    station_sel = station_dropdown.value
                    station_string = f'station {station_sel}'
                else:
                    station_string = 'all stations'
                    D[varnm_sel] = D[varnm_sel] + offset_value
    
                D[varnm_sel].attrs = var_attrs

                offset_metadata = f"Applied offset of {offset_value} [{units}] ({station_string})"
                
                if 'applied_offset' in D[varnm_sel].attrs:
                    D[varnm_sel].attrs['applied_offset'] += f'\n{offset_metadata}'
                else:
                    D[varnm_sel].attrs['applied_offset'] = offset_metadata
                    
                var_buttons.close()
                hbox_station.close()
                hbox_enter_value.close()
    
                with output_widget:
                    clear_output(wait=True)
                    print(f'-> {varnm_sel}: {offset_metadata}')

            else:
                var_buttons.close()
                hbox_station.close()
                hbox_enter_value.close()
    
                with output_widget:
                    clear_output(wait=True)
                    print(f' No offset value was entered -> Exited without changing anything. ')
                    
    def on_exit_button_click(change):
        if change['new']:
            
            var_buttons.close()
            hbox_station.close()
            hbox_enter_value.close()

            with output_widget:
                clear_output(wait=True)
                print(f' -> Exited without changing anything. ')

    apply_button.observe(on_apply_button_click, names='value')
    exit_button.observe(on_exit_button_click, names='value')

    # Create an HBox with station_buttons and station_dropdown
    hbox_enter_value = widgets.HBox([value_widget, apply_button, exit_button], layout=hbox_layout)

    
    display(var_buttons)
    display(hbox_station)
    display(hbox_enter_value)

    # Add an Output widget to capture the print statement when we are done
    
    output_widget = widgets.Output()
    display(output_widget)


In [190]:
apply_offset(D)

RadioButtons(description='Apply offset to:', options=('TEMP1', 'PTEMP1', 'CNDC1', 'PSAL1', 'SIGTH1', 'CHLA1_fl…

Output()

In [188]:
D

<xarray.Dataset>
Dimensions:             (PRES: 871, TIME: 10)
Coordinates:
  * PRES                (PRES) float64 3.0 4.0 5.0 6.0 ... 871.0 872.0 873.0
  * TIME                (TIME) float64 1.743e+04 1.743e+04 ... 1.743e+04
Data variables:
    TEMP1               (TIME, PRES) float64 nan 2.222e+05 2.222e+05 ... nan nan
    PTEMP1              (TIME, PRES) float64 nan 0.672 -0.3422 ... nan nan nan
    CNDC1               (TIME, PRES) float64 nan 50.13 49.35 ... nan nan nan
    PSAL1               (TIME, PRES) float64 nan 33.06 33.13 ... nan nan nan
    SIGTH1              (TIME, PRES) float64 nan 1.338e+03 1.338e+03 ... nan nan
    CHLA1_fluorescence  (TIME, PRES) float64 nan 1.597 1.749 ... nan nan nan
    SBE_FLAG            (TIME, PRES) float64 nan 0.0 0.0 0.0 ... nan nan nan nan
    STATION             (TIME) <U5 'AT280' 'AT281' 'AT282' ... 'AT288' 'AT289'
    LATITUDE            (TIME) float64 80.44 80.41 80.38 ... 80.04 80.03 80.02
    LONGITUDE           (TIME) float64 8.665 8.715 8.799 ... 10.75 10.9 11.0
    CRUISE              <U27 '!! CRUISE (Not assigned) !!'
Attributes:
    binned:          1 decibars (SBE software)
    history:         2017-09-24 to 2017-09-24: Data collection.\n2018-07-23: ...
    latitude:        80.0318
    longitude:       10.8973
    SBE_processing:  SBE SOFTWARE PROCESSING STEPS (extracted from .cnv file ...
    source_files:    E.g. AT288.HEX, AT288.XMLCON -> AT288.CNV
    featureType:     profile
    applied_offset:  TEMP1Applied offset of 1111.0 kg m-3 (all stations)

In [144]:
D.TEMP1

<xarray.DataArray 'TEMP1' (TIME: 10, PRES: 871)>
array([[     nan, 300.6721, 299.6579, ..., 299.7299, 299.7299, 299.7301],
       [302.687 , 303.3896, 303.3884, ...,      nan,      nan,      nan],
       [     nan, 303.7944, 303.7685, ...,      nan,      nan,      nan],
       ...,
       [306.0014, 306.0089, 306.0013, ...,      nan,      nan,      nan],
       [     nan, 305.7004, 305.7012, ...,      nan,      nan,      nan],
       [305.5639, 305.5616, 305.5671, ...,      nan,      nan,      nan]])
Coordinates:
  * PRES     (PRES) float64 3.0 4.0 5.0 6.0 7.0 ... 870.0 871.0 872.0 873.0
  * TIME     (TIME) float64 1.743e+04 1.743e+04 ... 1.743e+04 1.743e+04
Attributes:
    applied_offset:  Applied offset of 300.0 mS cm-1 (all stations)

In [96]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Assuming you have the _ctd_tools module imported and the _get_profile_variables function defined
# Also assuming that D is a dictionary containing station data

def apply_offset(D):

    vars = _ctd_tools._get_profile_variables(D)

    var_buttons = widgets.RadioButtons(
        options=vars,
        description='Apply offset to:',
        disabled=False
    )

    station_buttons = widgets.ToggleButtons(
        options=['All stations', 'Single station →'],
        description='Apply offset to:',
        disabled=False,
        button_style='',
        tooltips=['Apply an offset only to the profile from this particular station', 
                  'Apply an offset to every single profile in the dataset'],
    )

    value_textbox = widgets.Text(
        placeholder='Enter a numerical value',
        disabled=False   
    )
    value_label = widgets.Label(value=f'Value of offset:')
    value_widget = widgets.VBox([value_label, value_textbox])

    # Dropdown for selecting a single station
    station_dropdown = widgets.Dropdown(
        options=list(D['STATION'].values),  # Assuming keys are profile names
        value=D['STATION'].values[0],  # Set default value
        disabled=False,
        style={'description_width': 'initial'},  # Adjust width
    )
    
    # Align items to the flex-end to move the dropdown to the bottom of the HBox
    hbox_layout = widgets.Layout(align_items='flex-end')
    
    # Create an HBox with station_buttons and station_dropdown
    hbox_station = widgets.HBox([station_buttons, station_dropdown], layout=hbox_layout)

    apply_button = widgets.ToggleButton(
        value=False,
        description='Apply offset',
        disabled=False,
        button_style='success',
    )

    # Create an HBox with station_buttons and station_dropdown
    hbox_enter_value = widgets.HBox([value_widget, apply_button], layout=hbox_layout)

    display(var_buttons)
    display(hbox_station)
    display(hbox_enter_value)
    
    # Function to execute when the "Apply offset" button is clicked
    def on_apply_button_click(change):
        if change['new']:

            D.attrs['test'] = f"Apply offset of {value_textbox.value} to "
            
            if False:
                # Create additional widgets for confirmation
                confirmation_label = widgets.Label(value=f"Apply offset of {value_textbox.value} to:")
                confirm_button = widgets.Button(description='Confirm', button_style='success')
                exit_button = widgets.Button(description='Exit without changing', button_style='danger')
                confirmation_box = widgets.HBox([confirm_button, exit_button])
                
                # Function to execute when the "Confirm" button is clicked
                def on_confirm_button_click(b):
                    print("Confirmed! Do the offset operation here.")
                    clear_output()
                    # Additional code to perform offset operation
                    # Hide or remove the confirmation widgets
                    #confirm_button.close()
                    #exit_button.close()
                    #confirmation_label.close()
                    
                # Function to execute when the "Exit without changing" button is clicked
                def on_exit_button_click(b):
                    print("Exited without changing.")
                    clear_output()
                    # Hide or remove the confirmation widgets
                    #confirm_button.close()
                    #exit_button.close()
                    #confirmation_label.close()
                    
               #  def close_everything(self):
               #     fig = plt.gcf()
               #     fig.set_size_inches(0, 0)
               #     self.widgets_all.close()
               #     plt.close(fig)
    
    
                # Attach the on_confirm_button_click function to the button's on_click event
               # confirm_button.on_click(on_confirm_button_click)
                #exit_button.on_click(on_exit_button_click)
    
                # Display the confirmation widgets
                display(confirmation_label)
                display(confirmation_box)
    
    # Attach the on_apply_button_click function to the "Apply offset" button's observe method
    apply_button.observe(on_apply_button_click, names='value')

# Example usage
apply_offset(D)


RadioButtons(description='Apply offset to:', options=('TEMP1', 'PTEMP1', 'CNDC1', 'PSAL1', 'SIGTH1', 'CHLA1_fl…

_ctd_tools._